# NASSP Observational Cosmology Machine Learning Tutorial

## SIMTHEMBILE DLAMINI

First make sure the cell below executes without any errors. You should be running python 3. If you're not used to jupyter notebooks, have a look here https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.Dew0mVc

You need to write all your code here in this notebook and submit it as part of your solutions. Answer all the questions and copy the plots you make (with captions) into your solution pdf that you'll submit.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sncosmo
from astropy.table import Table
from iminuit import Minuit
from sklearn import metrics


# Section 1: Setup

 How to read in a supernova light curve as an astropy table

I've saved the supernova light curves in a format that `sncosmo` can read: an astropy table. The cell below reads in a single object's data and displays the table. 

The columns are: <br>
`mjd`: Time in days since the first observation of the supernova <br>
`filter`: Which filter band the observation is in (i.e. which colour)<br>
`flux`: The flux (brightness) in that band<br>
`flux_error`: Uncertainty on the flux due to instrumental noise<br>
`zp`: What magnitude the flux is calibrated to (don't worry too much about this)<br>
`zpsys`: The system used to calibrate the flux (also don't worry)

In [15]:
lc_file = 'training_data/34.dat'
lc = Table.read(lc_file, format='ascii')
lc

FileNotFoundError: [Errno 2] No such file or directory: 'training_data/34.dat'

# Section 2: Feature Extraction


## <font color='red'> Question 1a: Briefly describe how sncosmo's fit_lc function works.</font>

sncosmo.fit_lc function fit model parameters to data by minimizing chi^2. This function defines a chi^2 to minimize, makes initial guesses for t0 and amplitude, then runs a minimizer.

For t0 guess, if t0 is being fit and guess_t_0=True, the function will guess the initial starting point for t0 based on the data.

for amplitude guess, if amplitude (assumed to be the first model parameter) is being fit and guess_amplitude=True, the function will guess the initial starting point for the amplitude based on the data.

redshift guess: If redshift(z) is being fit and guess_z = True, the function will set the initial value of z to the average of the bounds on z.

## <font color='red'> Question 1b: Briefly describe what each of the 5 parameters in the SALT2 model is.</font>


z: is the redshift of the source.

t0: is the observer-frame time corresponding to the source's phase 0

x0: is a simple multiplicative scaling factor on the whole spectral timeseries

x1: SALT2 light-curve width

c: colour. SALT2 use colour parameter to get rid of dust.

### Use `sncosmo` to fit the SALT2 model to this light curve

In [16]:
def fit_supernova(lc):
    bnds = {'z':(0.01, 1.5), 't0':(-100,100),'x0':(-1e-3, 1e-3), 'x1':(-3, 3), 'c':(-0.5, 0.5)}
    mod = sncosmo.Model('salt2-extended')
    res = sncosmo.fit_lc(lc, mod, vparam_names=mod.param_names, bounds=bnds, minsnr=0)
    return res[0].parameters

In [17]:
prms  =fit_supernova(lc)
print('Best fitting SALT2 parameters: [z,t0,x0,x1,c]:')
print(prms)

NameError: name 'lc' is not defined

Check that the fitting worked correctly by using sncosmo's plot_lc function 

In [18]:
mod = sncosmo.Model('salt2-extended')
mod.parameters = prms
sncosmo.plot_lc(lc, mod)

ValueError: Incorrect number of parameters.

Now fit all 1000 light curves in the training data and save the parameters to file

These will be our features for the machine learning. Your final array should have dimensions [1000, 5] because there are 5 features (the SALT2 parameters). 

**NB** This will take around 30 minutes at least (unless you know how to parallelise it) so run this before the tutorial session and save these features to disk.

In [19]:
#Feature Extraction

i,prms,LC = 0,[],[]

while(i<1000):
    lc_file = 'training_data/'+str(i)+'.dat'
    lc = Table.read(lc_file, format='ascii')
    prms.append(fit_supernova(lc))
    LC.append(lc)
    i+=1
    

FileNotFoundError: [Errno 2] No such file or directory: 'training_data/0.dat'

<font color='red'> Question 2: Run plot_lc and make plots for data files 3, 4 and 5. Why do some of these fit badly? 

In [20]:
j=3
while(j<=5):
    mod = sncosmo.Model('salt2-extended')
    mod.parameters = prms[j]
    sncosmo.plot_lc(LC[j], mod)
    j+=1
    plt.show()
    
            

IndexError: list index out of range

## <font color='red'> Question 3: Plot histograms for the x1 and c parameters, colouring the Ia's and the non-Ia's different colours. Do you notice any differences between the two types?

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
id, type = np.loadtxt('training_labels.txt',skiprows=1,unpack=True)
type_1a_indeces = np.where(type == 1)
non_1a_indeces = np.where(type != 1)
id_1a = id[type_1a_indeces]
parameters = np.array(prms).transpose()
x1_data = parameters[3]
c_data = parameters[4]
plt.hist(x1_data[type_1a_indeces])
plt.hist(c_data[type_1a_indeces])
plt.title('Histogram of type_1a vs c_data')
plt.legend()
plt.show()
plt.hist(x1_data[non_1a_indeces])
plt.hist(c_data[non_1a_indeces])
plt.title('Histogram of non_type_1a vs c_data')
plt.legend()
plt.show()

y = np.copy(type)
y[non_1a_indeces] = 0
X = np.copy(prms)
#indxs = np.random.permutation(y.shape[0])
num_test = 300
#num_train = 1000 - num_test
#indxs_train, indxs_test = indxs[:num_train], indxs[num_train:]
#X_train, X_test = X[indxs_train], X[indxs_test]
#y_train, y_test = y[indxs_train], y[indxs_test]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,random_state=50)

FileNotFoundError: [Errno 2] No such file or directory: 'training_labels.txt'

# Section 3: Machine Learning with Scikit-Learn

You need to run 3 machine learning algorithms on your set of SALT2 features, at least ONE of which must NOT have been covered in the class (i.e. you may use a maximum of two of: k-nearest neighbours, neural networks and ensemble methods with decision trees)

Remember to divide up your labeled data into a training and a validation set so that you can evaluate your algorithms. 

## <font color='red'>Question 4: For each of the three algorithms you are using, answer the following:
1. Does the algorithm you're using require the features to be rescaled (http://scikit-learn.org/stable/modules/preprocessing.html )? If so, what did you use to rescale them?




2. What values of the hyperparameters of the algorithm did you use? How did you select them (consider investigating scikit-learn's `GridSearchCV` function)?




3. How much data are you using to evaluate the algorithms?

In [22]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2, algorithm='auto', p=1)
neigh.fit(X_train, y_train)
indxs = np.where(neigh.predict(X_test)-y_test != 0)
print(1.0 - indxs[0].shape[0]/float(num_test))
#print(indxs[0])

NameError: name 'X_train' is not defined

 KNN:really does not need to rescale.n_neighbors which is the number of neighborhood.
 In k-NN classification, the output is a class membership. An object is classified by a majority vote of its neighbors,        with the object being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). 
If k = 1, then the object is simply assigned to the class of that single nearest neighbor.
hyperparameters : n_neighbors=2, algorithm='auto', p=1
training was 70% of the total data
test was 30% of the total data.

In [23]:
from sklearn.neural_network import MLPClassifier
mcl = MLPClassifier(solver='adam',activation='tanh', hidden_layer_sizes = (100,100,100),tol=1e-11, max_iter = 3000, alpha=1e-5, random_state=10, learning_rate = 'invscaling')
mcl.fit(X_train, y_train)
ind = np.where(mcl.predict(X_test)-y_test != 0)
print(1.0 - ind[0].shape[0]/float(num_test))
    
    
    

NameError: name 'X_train' is not defined

MLPClassifier: it does require rescaling of a data of 0-1 scale. 
MLPClassifier trains iteratively since at each time step the partial derivatives of the loss function with respect to the model parameters are computed to update the parameters. 
    
hyperparameters : solver='adam',activation='tanh', hidden_layer_sizes = (100,100,100),tol=1e-11, max_iter = 3000, alpha=1e-5, random_state=10, learning_rate = 'invscaling'
    
training was 70% of the total data
test was 30% of the total data.   

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
ind = np.where(gnb.predict(X_test)-y_test != 0)
print(1.0 - ind[0].shape[0]/float(num_test))

NameError: name 'X_train' is not defined

GaussianNB:it does feature scaling by design but it does not have an effect if you perfoming one manual.
           we used the size of data to rescaling  in this Algorithm.
    
hyperparameters : class_prior_ : array, shape (n_classes,)

                                probability of each class.

                class_count_ : array, shape (n_classes,)

                number of training samples observed in each class.

                theta_ : array, shape (n_classes, n_features)

                           mean of each feature per class

                sigma_ : array, shape (n_classes, n_features)
 
                             variance of each feature per class
       
        
        
training was 70% of the total data
test was 30% of the total data.       

# Section 4: Evaluating the Classification

For this section you'll need the `roc_curve` and `predict_proba` functions from sklearn.

## <font color='red'> Question 5: On one figure, plot the ROC curve for each of your classifiers and include the name of the classifier and the calculated AUC in the legend.

In [12]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
y_score = []
y_score.append(mcl.fit(X_train, y_train).predict(X_test))
y_score.append(neigh.fit(X_train,y_train).predict(X_test))
y_score.append(gnb.fit(X_train,y_train).predict(X_test))
classifiers = ['MLP','KNN','GNB']

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_score[i])
    roc_auc = auc(fpr[i], tpr[i])
    plt.plot(fpr[i],tpr[i],label=classifiers[i])
plt.legend()
plt.show()
print(roc_auc)

NameError: name 'X_train' is not defined

# Section 5: Test Data Analysis

First, repeat the above procedure to extract features from the test data. Remember to save your features to disk so you don't have to rerun it!

Next, use your best _trained classifier_ to predict the probabilities of the test objects (use `predict_proba` function of your classifier). Save these to a text file, that will look like this:<br>

`#ID    prob(Ia)    prob(II)    prob(Ibc)` <br>
`1000   0.87        0.03        0.1 `    <br>
`1001   0.25        0.04        0.79  `  <br>
`...`

In [13]:
#Feature Extraction

i,prms,LC = 0,[],[]

while(i<1000):
    lc_file = 'test_data/'+str(i)+'.dat'
    lc = Table.read(lc_file, format='ascii')
    prms.append(fit_supernova(lc))
    LC.append(lc)
    i+=1

y = np.copy(type)
X = np.copy(prms)

mcl = MLPClassifier(solver='adam',activation='tanh', hidden_layer_sizes = (100,100,100),tol=1e-11, max_iter = 3000, alpha=1e-5, random_state=10, learning_rate = 'invscaling')
mcl.fit(X, y)
datafile = open("file.txt", 'w')

datafile.write('#ID\tprob(Ia)\tprob(II)\tprob(Ibc)\n')
pred = mcl.predict_proba(X)
i = 0
for prediction in pred:
    i += 1
    datafile.write('%d\t%f\t%f\t%f\n'%(i,prediction[0],prediction[1],prediction[2]))
    
datafile.close()


FileNotFoundError: [Errno 2] No such file or directory: 'test_data/0.dat'